# Get the Emails from Pickled DF

In [58]:
import yaml
import pickle

# Open the pickle file and load the DataFrame
with open('../.pickles/df_emails_imap.pkl', 'rb') as f:
    df_emails_imap = pickle.load(f)

In [59]:
from IPython.display import display

display(df_emails_imap)

,Date,Subject,From,To,Message-ID,Body,Reply-To
0,2024-12-01 08:19:12+00:00,See what's streaming in December,"""IMDb.com"" <do-not-reply@imdb.com>",mike.cancell@gmail.com,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,IMDb Streaming Guides\r\nCheck out our watch g...,"""IMDb.com"" <do-not-reply@imdb.com>"
1,2024-12-01 08:51:40+00:00,Coming soon to State Theatre New Jersey: Cirqu...,Bandsintown <notifications@updates.bandsintown...,mike.cancell@gmail.com,<msg-19742805299@email.bandsintown.com>,"Just Announced Cirque Musica, New Jersey Symph...",Bandsintown <notifications@updates.bandsintown...
2,2024-12-01 12:03:31+00:00,=?utf-8?q?Will_Obama_go_to_jail=3F?=,Quora Digest <english-personalized-digest@quor...,mike.cancell@gmail.com,<LARkCC7mJXguO-fXuuzZcwMdZJYRZCZHc2nstcrUwcMNj...,Top stories for Mike\r\n\r\n-----\r\n\r\nQuest...,Quora Digest <english-personalized-digest@quor...
3,2024-12-01 12:27:32+00:00,"Your Daily Digest for Sun, 12/1 is ready to view",USPS Informed Delivery <USPSInformeddelivery@e...,mike.cancell@gmail.com,<20241201122732.9884238d69c03837@email.informe...,Content-Type: text/plain; charset=UTF-8\r\nCon...,USPS Informed Delivery <USPSInformeddelivery@e...
4,2024-12-01 13:10:38+00:00,Your Weekly Ad + Holiday Savings to Deck the H...,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?= <extracare@yo...,mike@mikecancell.com,<0.3.9.54A.1DB43F26A5DD630.0@omp.your.cvs.com>,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?= <noreply@your...
...,...,...,...,...,...,...,...
1121,2025-01-27 18:31:14+00:00,RE: Interactions - Michael Cancell,Benemax Service <Benemax.service@onedigital.com>,"""'mike@mikecancell.com'"" <mike@mikecancell.com>",<SN7PR20MB5920F391F342A45235E3FF4097EC2@SN7PR2...,"Hi Mike,\r\n\r\nThe Rx reimbursement processed...",Benemax Service <Benemax.service@onedigital.com>
1122,2025-01-27 19:51:19+00:00,Update from Deque Systems,Deque Systems <recruiting+335397270@applytojob...,Michael Cancell <mike@mikecancell.com>,<c449ad01-c532-4f18-9ceb-5d415e295c10@mtasv.net>,"<p>Hello Michael,</p><p>Thank you for the time...",Deque Systems <c202501271951190SHQYAUUAJHU5CCE...
1123,2025-01-27 20:18:01+00:00,Thank you for applying,Amazon Web Services <aws-marketing-email-repli...,mike.cancell@gmail.com,<01000194a969f744-85025198-21e5-4440-a268-1c22...,Thank you for your interest in the $300 AWS cr...,aws-marketing-email-replies@amazon.com
1124,2025-01-27 20:18:30+00:00,"Follow your application status, Mike",Siemens Talent Acquisition <talentacquisition....,mike.cancell@gmail.com,<u6w8VwXDpwNOIRXHsa0jc4qtKza1Awt3@siemens.avat...,"Hi Mike,\r\n\r\nThank you for your interest in...",Siemens Talent Acquisition <talentacquisition....


## Convert the Email Date to a Valid Timestamp

In [60]:
import pandas as pd
# Convert 'Date' column to datetime format
df_emails_imap['Timestamp'] = pd.to_datetime(df_emails_imap['Date'])

# Drop the old 'Date' column
df_emails_imap.drop(columns=['Date'], inplace=True)

# Display the updated DataFrame
display(df_emails_imap)

,Subject,From,To,Message-ID,Body,Reply-To,Timestamp
0,See what's streaming in December,"""IMDb.com"" <do-not-reply@imdb.com>",mike.cancell@gmail.com,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,IMDb Streaming Guides\r\nCheck out our watch g...,"""IMDb.com"" <do-not-reply@imdb.com>",2024-12-01 08:19:12+00:00
1,Coming soon to State Theatre New Jersey: Cirqu...,Bandsintown <notifications@updates.bandsintown...,mike.cancell@gmail.com,<msg-19742805299@email.bandsintown.com>,"Just Announced Cirque Musica, New Jersey Symph...",Bandsintown <notifications@updates.bandsintown...,2024-12-01 08:51:40+00:00
2,=?utf-8?q?Will_Obama_go_to_jail=3F?=,Quora Digest <english-personalized-digest@quor...,mike.cancell@gmail.com,<LARkCC7mJXguO-fXuuzZcwMdZJYRZCZHc2nstcrUwcMNj...,Top stories for Mike\r\n\r\n-----\r\n\r\nQuest...,Quora Digest <english-personalized-digest@quor...,2024-12-01 12:03:31+00:00
3,"Your Daily Digest for Sun, 12/1 is ready to view",USPS Informed Delivery <USPSInformeddelivery@e...,mike.cancell@gmail.com,<20241201122732.9884238d69c03837@email.informe...,Content-Type: text/plain; charset=UTF-8\r\nCon...,USPS Informed Delivery <USPSInformeddelivery@e...,2024-12-01 12:27:32+00:00
4,Your Weekly Ad + Holiday Savings to Deck the H...,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?= <extracare@yo...,mike@mikecancell.com,<0.3.9.54A.1DB43F26A5DD630.0@omp.your.cvs.com>,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?= <noreply@your...,2024-12-01 13:10:38+00:00
...,...,...,...,...,...,...,...
1121,RE: Interactions - Michael Cancell,Benemax Service <Benemax.service@onedigital.com>,"""'mike@mikecancell.com'"" <mike@mikecancell.com>",<SN7PR20MB5920F391F342A45235E3FF4097EC2@SN7PR2...,"Hi Mike,\r\n\r\nThe Rx reimbursement processed...",Benemax Service <Benemax.service@onedigital.com>,2025-01-27 18:31:14+00:00
1122,Update from Deque Systems,Deque Systems <recruiting+335397270@applytojob...,Michael Cancell <mike@mikecancell.com>,<c449ad01-c532-4f18-9ceb-5d415e295c10@mtasv.net>,"<p>Hello Michael,</p><p>Thank you for the time...",Deque Systems <c202501271951190SHQYAUUAJHU5CCE...,2025-01-27 19:51:19+00:00
1123,Thank you for applying,Amazon Web Services <aws-marketing-email-repli...,mike.cancell@gmail.com,<01000194a969f744-85025198-21e5-4440-a268-1c22...,Thank you for your interest in the $300 AWS cr...,aws-marketing-email-replies@amazon.com,2025-01-27 20:18:01+00:00
1124,"Follow your application status, Mike",Siemens Talent Acquisition <talentacquisition....,mike.cancell@gmail.com,<u6w8VwXDpwNOIRXHsa0jc4qtKza1Awt3@siemens.avat...,"Hi Mike,\r\n\r\nThank you for your interest in...",Siemens Talent Acquisition <talentacquisition....,2025-01-27 20:18:30+00:00


## From the From and To Cols, Extract the Actual Addr and the Display Name in Sep Cols

In [61]:
import re

# Function to extract email address and display name
def extract_email_and_name(email):
    match = re.match(r'(?:"?([^"]*)"?\s)?(?:<?(.+@[^>]+)>?)', email)
    if match:
        display_name, email_address = match.groups()
        if not display_name:
            display_name = email_address.split('@')[0]
        return display_name, email_address
    return email, email

# Apply the function to the 'From' column
df_emails_imap[['From_Display_Name', 'From_Addr']] = df_emails_imap['From'].apply(lambda x: pd.Series(extract_email_and_name(x)))

# Apply the function to the 'To' column
df_emails_imap[['To_Display_Name', 'To_Addr']] = df_emails_imap['To'].apply(lambda x: pd.Series(extract_email_and_name(x)))

# Apply the function to the 'Reply-To' column
df_emails_imap[['Reply-To_Display_Name', 'Reply-To_Addr']] = df_emails_imap['Reply-To'].apply(lambda x: pd.Series(extract_email_and_name(x)))
#
display(df_emails_imap[['Timestamp', 'Subject', 'From_Display_Name', 'From_Addr', 'To_Display_Name', 'To_Addr', 'Message-ID', 'Body', 'Reply-To']])

,Timestamp,Subject,From_Display_Name,From_Addr,To_Display_Name,To_Addr,Message-ID,Body,Reply-To
0,2024-12-01 08:19:12+00:00,See what's streaming in December,IMDb.com,do-not-reply@imdb.com,mike.cancell,mike.cancell@gmail.com,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,IMDb Streaming Guides\r\nCheck out our watch g...,"""IMDb.com"" <do-not-reply@imdb.com>"
1,2024-12-01 08:51:40+00:00,Coming soon to State Theatre New Jersey: Cirqu...,Bandsintown,notifications@updates.bandsintown.com,mike.cancell,mike.cancell@gmail.com,<msg-19742805299@email.bandsintown.com>,"Just Announced Cirque Musica, New Jersey Symph...",Bandsintown <notifications@updates.bandsintown...
2,2024-12-01 12:03:31+00:00,=?utf-8?q?Will_Obama_go_to_jail=3F?=,Quora Digest,english-personalized-digest@quora.com,mike.cancell,mike.cancell@gmail.com,<LARkCC7mJXguO-fXuuzZcwMdZJYRZCZHc2nstcrUwcMNj...,Top stories for Mike\r\n\r\n-----\r\n\r\nQuest...,Quora Digest <english-personalized-digest@quor...
3,2024-12-01 12:27:32+00:00,"Your Daily Digest for Sun, 12/1 is ready to view",USPS Informed Delivery,USPSInformeddelivery@email.informeddelivery.us...,mike.cancell,mike.cancell@gmail.com,<20241201122732.9884238d69c03837@email.informe...,Content-Type: text/plain; charset=UTF-8\r\nCon...,USPS Informed Delivery <USPSInformeddelivery@e...
4,2024-12-01 13:10:38+00:00,Your Weekly Ad + Holiday Savings to Deck the H...,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,extracare@your.cvs.com,mike,mike@mikecancell.com,<0.3.9.54A.1DB43F26A5DD630.0@omp.your.cvs.com>,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?= <noreply@your...
...,...,...,...,...,...,...,...,...,...
1121,2025-01-27 18:31:14+00:00,RE: Interactions - Michael Cancell,Benemax Service,Benemax.service@onedigital.com,'mike@mikecancell.com',mike@mikecancell.com,<SN7PR20MB5920F391F342A45235E3FF4097EC2@SN7PR2...,"Hi Mike,\r\n\r\nThe Rx reimbursement processed...",Benemax Service <Benemax.service@onedigital.com>
1122,2025-01-27 19:51:19+00:00,Update from Deque Systems,Deque Systems,recruiting+335397270@applytojob.com,Michael Cancell,mike@mikecancell.com,<c449ad01-c532-4f18-9ceb-5d415e295c10@mtasv.net>,"<p>Hello Michael,</p><p>Thank you for the time...",Deque Systems <c202501271951190SHQYAUUAJHU5CCE...
1123,2025-01-27 20:18:01+00:00,Thank you for applying,Amazon Web Services,aws-marketing-email-replies@amazon.com,mike.cancell,mike.cancell@gmail.com,<01000194a969f744-85025198-21e5-4440-a268-1c22...,Thank you for your interest in the $300 AWS cr...,aws-marketing-email-replies@amazon.com
1124,2025-01-27 20:18:30+00:00,"Follow your application status, Mike",Siemens Talent Acquisition,talentacquisition.hr@siemens.com,mike.cancell,mike.cancell@gmail.com,<u6w8VwXDpwNOIRXHsa0jc4qtKza1Awt3@siemens.avat...,"Hi Mike,\r\n\r\nThank you for your interest in...",Siemens Talent Acquisition <talentacquisition....


## Quick Check for Missing Data

In [62]:
missing_data = df_emails_imap.isnull().sum()
print(missing_data)

Subject                  0
From                     0
To                       0
Message-ID               0
Body                     0
Reply-To                 0
Timestamp                0
From_Display_Name        0
From_Addr                0
To_Display_Name          0
To_Addr                  0
Reply-To_Display_Name    0
Reply-To_Addr            0
dtype: int64


## Clean up the Body to remove HTML & CSS Markup

In [63]:
from bs4 import BeautifulSoup
import re

# Function to remove HTML markup, CSS style tags, and URLs
def remove_markup_and_urls(text):
    soup = BeautifulSoup(text, 'html.parser')
    # Remove CSS style tags
    for style in soup(["style"]):
        style.decompose()
    text_without_markup = soup.get_text()
    text_without_urls = re.sub(r'http\S+|www\S+|https\S+', '', text_without_markup, flags=re.MULTILINE)
    # Remove any remaining CSS markup
    text_without_css = re.sub(r'{.*?}', '', text_without_urls, flags=re.DOTALL)
    # Remove any remaining HTML tags
    clean_text = re.sub(r'<.*?>', '', text_without_css)
    return clean_text.strip()

# Apply the function to the 'Body' column
df_emails_imap['Body_Text'] = df_emails_imap['Body'].apply(remove_markup_and_urls)

# Display the updated DataFrame
display(df_emails_imap)

C:\Users\mike\AppData\Local\Temp\ipykernel_29012\3308429135.py:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\mike\AppData\Local\Temp\ipykernel_29012\3308429135.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


,Subject,From,To,Message-ID,Body,Reply-To,Timestamp,From_Display_Name,From_Addr,To_Display_Name,To_Addr,Reply-To_Display_Name,Reply-To_Addr,Body_Text
0,See what's streaming in December,"""IMDb.com"" <do-not-reply@imdb.com>",mike.cancell@gmail.com,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,IMDb Streaming Guides\r\nCheck out our watch g...,"""IMDb.com"" <do-not-reply@imdb.com>",2024-12-01 08:19:12+00:00,IMDb.com,do-not-reply@imdb.com,mike.cancell,mike.cancell@gmail.com,IMDb.com,do-not-reply@imdb.com,IMDb Streaming Guides\r\nCheck out our watch g...
1,Coming soon to State Theatre New Jersey: Cirqu...,Bandsintown <notifications@updates.bandsintown...,mike.cancell@gmail.com,<msg-19742805299@email.bandsintown.com>,"Just Announced Cirque Musica, New Jersey Symph...",Bandsintown <notifications@updates.bandsintown...,2024-12-01 08:51:40+00:00,Bandsintown,notifications@updates.bandsintown.com,mike.cancell,mike.cancell@gmail.com,Bandsintown,notifications@updates.bandsintown.com,"Just Announced Cirque Musica, New Jersey Symph..."
2,=?utf-8?q?Will_Obama_go_to_jail=3F?=,Quora Digest <english-personalized-digest@quor...,mike.cancell@gmail.com,<LARkCC7mJXguO-fXuuzZcwMdZJYRZCZHc2nstcrUwcMNj...,Top stories for Mike\r\n\r\n-----\r\n\r\nQuest...,Quora Digest <english-personalized-digest@quor...,2024-12-01 12:03:31+00:00,Quora Digest,english-personalized-digest@quora.com,mike.cancell,mike.cancell@gmail.com,Quora Digest,english-personalized-digest@quora.com,Top stories for Mike\r\n\r\n-----\r\n\r\nQuest...
3,"Your Daily Digest for Sun, 12/1 is ready to view",USPS Informed Delivery <USPSInformeddelivery@e...,mike.cancell@gmail.com,<20241201122732.9884238d69c03837@email.informe...,Content-Type: text/plain; charset=UTF-8\r\nCon...,USPS Informed Delivery <USPSInformeddelivery@e...,2024-12-01 12:27:32+00:00,USPS Informed Delivery,USPSInformeddelivery@email.informeddelivery.us...,mike.cancell,mike.cancell@gmail.com,USPS Informed Delivery,USPSInformeddelivery@email.informeddelivery.us...,Content-Type: text/plain; charset=UTF-8\r\nCon...
4,Your Weekly Ad + Holiday Savings to Deck the H...,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?= <extracare@yo...,mike@mikecancell.com,<0.3.9.54A.1DB43F26A5DD630.0@omp.your.cvs.com>,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?= <noreply@your...,2024-12-01 13:10:38+00:00,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,extracare@your.cvs.com,mike,mike@mikecancell.com,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,noreply@your.cvs.com,CVS\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,RE: Interactions - Michael Cancell,Benemax Service <Benemax.service@onedigital.com>,"""'mike@mikecancell.com'"" <mike@mikecancell.com>",<SN7PR20MB5920F391F342A45235E3FF4097EC2@SN7PR2...,"Hi Mike,\r\n\r\nThe Rx reimbursement processed...",Benemax Service <Benemax.service@onedigital.com>,2025-01-27 18:31:14+00:00,Benemax Service,Benemax.service@onedigital.com,'mike@mikecancell.com',mike@mikecancell.com,Benemax Service,Benemax.service@onedigital.com,"Hi Mike,\r\n\r\nThe Rx reimbursement processed..."
1122,Update from Deque Systems,Deque Systems <recruiting+335397270@applytojob...,Michael Cancell <mike@mikecancell.com>,<c449ad01-c532-4f18-9ceb-5d415e295c10@mtasv.net>,"<p>Hello Michael,</p><p>Thank you for the time...",Deque Systems <c202501271951190SHQYAUUAJHU5CCE...,2025-01-27 19:51:19+00:00,Deque Systems,recruiting+335397270@applytojob.com,Michael Cancell,mike@mikecancell.com,Deque Systems,c202501271951190SHQYAUUAJHU5CCE@mail.applytojo...,"Hello Michael,Thank you for the time and energ..."
1123,Thank you for applying,Amazon Web Services <aws-marketing-email-repli...,mike.cancell@gmail.com,<01000194a969f744-85025198-21e5-4440-a268-1c22...,Thank you for your interest in the $300 AWS cr...,aws-marketing-email-replies@amazon.com,2025-01-27 20:18:01+00:00,Amazon Web Services,aws-marketing-email-replies@amazon.com,mike.cancell,mike.cancell@gmail.com,aws-marketing-email-replies,aws-mar

In [64]:
print(df_emails_imap.dtypes)

Subject                               object
From                                  object
To                                    object
Message-ID                            object
Body                                  object
Reply-To                              object
Timestamp                datetime64[ns, UTC]
From_Display_Name                     object
From_Addr                             object
To_Display_Name                       object
To_Addr                               object
Reply-To_Display_Name                 object
Reply-To_Addr                         object
Body_Text                             object
dtype: object


## Drop the Original From/To Cols (post split)

In [65]:
# Drop the specified columns
df_emails_imap.drop(columns=['From', 'To', 'Body', 'Reply-To'], inplace=True)

# Display the remaining columns
display(df_emails_imap)

,Subject,Message-ID,Timestamp,From_Display_Name,From_Addr,To_Display_Name,To_Addr,Reply-To_Display_Name,Reply-To_Addr,Body_Text
0,See what's streaming in December,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,2024-12-01 08:19:12+00:00,IMDb.com,do-not-reply@imdb.com,mike.cancell,mike.cancell@gmail.com,IMDb.com,do-not-reply@imdb.com,IMDb Streaming Guides\r\nCheck out our watch g...
1,Coming soon to State Theatre New Jersey: Cirqu...,<msg-19742805299@email.bandsintown.com>,2024-12-01 08:51:40+00:00,Bandsintown,notifications@updates.bandsintown.com,mike.cancell,mike.cancell@gmail.com,Bandsintown,notifications@updates.bandsintown.com,"Just Announced Cirque Musica, New Jersey Symph..."
2,=?utf-8?q?Will_Obama_go_to_jail=3F?=,<LARkCC7mJXguO-fXuuzZcwMdZJYRZCZHc2nstcrUwcMNj...,2024-12-01 12:03:31+00:00,Quora Digest,english-personalized-digest@quora.com,mike.cancell,mike.cancell@gmail.com,Quora Digest,english-personalized-digest@quora.com,Top stories for Mike\r\n\r\n-----\r\n\r\nQuest...
3,"Your Daily Digest for Sun, 12/1 is ready to view",<20241201122732.9884238d69c03837@email.informe...,2024-12-01 12:27:32+00:00,USPS Informed Delivery,USPSInformeddelivery@email.informeddelivery.us...,mike.cancell,mike.cancell@gmail.com,USPS Informed Delivery,USPSInformeddelivery@email.informeddelivery.us...,Content-Type: text/plain; charset=UTF-8\r\nCon...
4,Your Weekly Ad + Holiday Savings to Deck the H...,<0.3.9.54A.1DB43F26A5DD630.0@omp.your.cvs.com>,2024-12-01 13:10:38+00:00,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,extracare@your.cvs.com,mike,mike@mikecancell.com,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,noreply@your.cvs.com,CVS\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
...,...,...,...,...,...,...,...,...,...,...
1121,RE: Interactions - Michael Cancell,<SN7PR20MB5920F391F342A45235E3FF4097EC2@SN7PR2...,2025-01-27 18:31:14+00:00,Benemax Service,Benemax.service@onedigital.com,'mike@mikecancell.com',mike@mikecancell.com,Benemax Service,Benemax.service@onedigital.com,"Hi Mike,\r\n\r\nThe Rx reimbursement processed..."
1122,Update from Deque Systems,<c449ad01-c532-4f18-9ceb-5d415e295c10@mtasv.net>,2025-01-27 19:51:19+00:00,Deque Systems,recruiting+335397270@applytojob.com,Michael Cancell,mike@mikecancell.com,Deque Systems,c202501271951190SHQYAUUAJHU5CCE@mail.applytojo...,"Hello Michael,Thank you for the time and energ..."
1123,Thank you for applying,<01000194a969f744-85025198-21e5-4440-a268-1c22...,2025-01-27 20:18:01+00:00,Amazon Web Services,aws-marketing-email-replies@amazon.com,mike.cancell,mike.cancell@gmail.com,aws-marketing-email-replies,aws-marketing-email-replies@amazon.com,Thank you for your interest in the $300 AWS cr...
1124,"Follow your application status, Mike",<u6w8VwXDpwNOIRXHsa0jc4qtKza1Awt3@siemens.avat...,2025-01-27 20:18:30+00:00,Siemens Talent Acquisition,talentacquisition.hr@siemens.com,mike.cancell,mike.cancell@gmail.com,Siemens Talent Acquisition,talentacquisition.hr@siemens.com,"Hi Mike,\r\n\r\nThank you for your interest in..."


## Create a Category from the Body and From Addr

In [66]:
# Define a function to categorize the body text
def categorize_body_text(body_text):
    body_text_lower = body_text.lower()
    if 'rejection' in body_text_lower or 'not selected' in body_text_lower or 'unfortunately' in body_text_lower:
        return 'Job Rejection'
    elif 'application' in body_text_lower and 'received' in body_text_lower:
        return 'Job Application'
    elif 'invoice' in body_text_lower:
        return 'Invoice'
    elif 'promotion' in body_text_lower:
        return 'Promotion'
    elif 'newsletter' in body_text_lower:
        return 'Newsletter'
    elif 'reminder' in body_text_lower:
        return 'Reminder'
    elif 'meeting' in body_text_lower or 'schedule' in body_text_lower:
        return 'Meeting'
    elif 'password' in body_text_lower or 'reset' in body_text_lower:
        return 'Password Reset'
    elif 'shipping' in body_text_lower or 'delivery' in body_text_lower:
        return 'Shipping'
    elif 'receipt' in body_text_lower or 'purchase' in body_text_lower:
        return 'Receipt'
    else:
        return 'Other'

# Apply the function to the 'Body_Text' column
df_emails_imap['Category'] = df_emails_imap['Body_Text'].apply(categorize_body_text)

# Function to categorize by From_Addr
def categorize_from_addr(from_addr):
    if 'wellsfargo' in from_addr:
        return 'Bank'
    elif 'greenhouse' in from_addr or 'lever' in from_addr:
        return 'Job Application'
    elif 'usps' in from_addr:
        return 'Shipping'
    elif 'njmmail' in from_addr:
        return 'Insurance'
    elif 'imdb' in from_addr:
        return 'Entertainment'
    elif 'bandsintown' in from_addr:
        return 'Music'
    elif 'quora' in from_addr:
        return 'Newsletter'
    elif 'synchronyfinancial' in from_addr:
        return 'Finance'
    elif 'google' in from_addr:
        return 'Tech'
    elif 'dayforce' in from_addr or 'trimble' in from_addr or 'workday' in from_addr:
        return 'Job Application'
    elif 'chick-fil-a' in from_addr:
        return 'Food'
    else:
        return 'Other'

# Apply the function to the 'From_Addr' column
df_emails_imap['From_Addr_Category'] = df_emails_imap['From_Addr'].apply(categorize_from_addr)

# Combine the categories from Body_Text and From_Addr
def combine_categories(row):
    if row['Category'] != 'Other':
        return row['Category']
    else:
        return row['From_Addr_Category']

# Apply the function to combine categories
df_emails_imap['Email_Category'] = df_emails_imap.apply(combine_categories, axis=1)

# Drop the intermediate 'From_Addr_Category' column
df_emails_imap.drop(columns=['From_Addr_Category'], inplace=True)

# Display the updated DataFrame
display(df_emails_imap)

,Subject,Message-ID,Timestamp,From_Display_Name,From_Addr,To_Display_Name,To_Addr,Reply-To_Display_Name,Reply-To_Addr,Body_Text,Category,Email_Category
0,See what's streaming in December,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,2024-12-01 08:19:12+00:00,IMDb.com,do-not-reply@imdb.com,mike.cancell,mike.cancell@gmail.com,IMDb.com,do-not-reply@imdb.com,IMDb Streaming Guides\r\nCheck out our watch g...,Other,Entertainment
1,Coming soon to State Theatre New Jersey: Cirqu...,<msg-19742805299@email.bandsintown.com>,2024-12-01 08:51:40+00:00,Bandsintown,notifications@updates.bandsintown.com,mike.cancell,mike.cancell@gmail.com,Bandsintown,notifications@updates.bandsintown.com,"Just Announced Cirque Musica, New Jersey Symph...",Other,Music
2,=?utf-8?q?Will_Obama_go_to_jail=3F?=,<LARkCC7mJXguO-fXuuzZcwMdZJYRZCZHc2nstcrUwcMNj...,2024-12-01 12:03:31+00:00,Quora Digest,english-personalized-digest@quora.com,mike.cancell,mike.cancell@gmail.com,Quora Digest,english-personalized-digest@quora.com,Top stories for Mike\r\n\r\n-----\r\n\r\nQuest...,Other,Newsletter
3,"Your Daily Digest for Sun, 12/1 is ready to view",<20241201122732.9884238d69c03837@email.informe...,2024-12-01 12:27:32+00:00,USPS Informed Delivery,USPSInformeddelivery@email.informeddelivery.us...,mike.cancell,mike.cancell@gmail.com,USPS Informed Delivery,USPSInformeddelivery@email.informeddelivery.us...,Content-Type: text/plain; charset=UTF-8\r\nCon...,Shipping,Shipping
4,Your Weekly Ad + Holiday Savings to Deck the H...,<0.3.9.54A.1DB43F26A5DD630.0@omp.your.cvs.com>,2024-12-01 13:10:38+00:00,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,extracare@your.cvs.com,mike,mike@mikecancell.com,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,noreply@your.cvs.com,CVS\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,Shipping,Shipping
...,...,...,...,...,...,...,...,...,...,...,...,...
1121,RE: Interactions - Michael Cancell,<SN7PR20MB5920F391F342A45235E3FF4097EC2@SN7PR2...,2025-01-27 18:31:14+00:00,Benemax Service,Benemax.service@onedigital.com,'mike@mikecancell.com',mike@mikecancell.com,Benemax Service,Benemax.service@onedigital.com,"Hi Mike,\r\n\r\nThe Rx reimbursement processed...",Job Application,Job Application
1122,Update from Deque Systems,<c449ad01-c532-4f18-9ceb-5d415e295c10@mtasv.net>,2025-01-27 19:51:19+00:00,Deque Systems,recruiting+335397270@applytojob.com,Michael Cancell,mike@mikecancell.com,Deque Systems,c202501271951190SHQYAUUAJHU5CCE@mail.applytojo...,"Hello Michael,Thank you for the time and energ...",Other,Other
1123,Thank you for applying,<01000194a969f744-85025198-21e5-4440-a268-1c22...,2025-01-27 20:18:01+00:00,Amazon Web Services,aws-marketing-email-replies@amazon.com,mike.cancell,mike.cancell@gmail.com,aws-marketing-email-replies,aws-marketing-email-replies@amazon.com,Thank you for your interest in the $300 AWS cr...,Job Rejection,Job Rejection
1124,"Follow your application status, Mike",<u6w8VwXDpwNOIRXHsa0jc4qtKza1Awt3@siemens.avat...,2025-01-27 20:18:30+00:00,Siemens Talent Acquisition,talentacquisition.hr@siemens.com,mike.cancell,mike.cancell@gmail.com,Siemens Talent Acquisition,talentacquisition.hr@siemens.com,"Hi Mike,\r\n\r\nThank you for your interest in...",Other,Other


# Design a Schema for the Data

### Defined a Date_Time Dimension

In [67]:
# Create a datetime dimension DataFrame
df_datetime_dim = pd.DataFrame({
	'fk_Timestamp': df_emails_imap['Timestamp'].unique()
})

# Add additional columns
df_datetime_dim['Date'] = df_datetime_dim['fk_Timestamp'].dt.date
df_datetime_dim['Time'] = df_datetime_dim['fk_Timestamp'].dt.time
df_datetime_dim['Hour'] = df_datetime_dim['fk_Timestamp'].dt.hour
df_datetime_dim['Day_of_Week'] = df_datetime_dim['fk_Timestamp'].dt.dayofweek
df_datetime_dim['Day_of_Week_String'] = df_datetime_dim['fk_Timestamp'].dt.strftime('%a')

# Arrange 'fk_Timestamp' as the first column
cols = ['fk_Timestamp'] + [col for col in df_datetime_dim.columns if col != 'fk_Timestamp']
df_datetime_dim = df_datetime_dim[cols]

# Display the updated DataFrame
display(df_datetime_dim)


,fk_Timestamp,Date,Time,Hour,Day_of_Week,Day_of_Week_String
0,2024-12-01 08:19:12+00:00,2024-12-01,08:19:12,8,6,Sun
1,2024-12-01 08:51:40+00:00,2024-12-01,08:51:40,8,6,Sun
2,2024-12-01 12:03:31+00:00,2024-12-01,12:03:31,12,6,Sun
3,2024-12-01 12:27:32+00:00,2024-12-01,12:27:32,12,6,Sun
4,2024-12-01 13:10:38+00:00,2024-12-01,13:10:38,13,6,Sun
...,...,...,...,...,...,...
1100,2025-01-27 18:31:14+00:00,2025-01-27,18:31:14,18,0,Mon
1101,2025-01-27 19:51:19+00:00,2025-01-27,19:51:19,19,0,Mon
1102,2025-01-27 20:18:01+00:00,2025-01-27,20:18:01,20,0,Mon
1103,2025-01-27 20:18:30+00:00,2025-01-27,20:18:30,20,0,Mon


## Create an Email Addresses Dim

In [68]:
# Extract unique 'From' addresses and display names
from_addresses = df_emails_imap[['From_Display_Name', 'From_Addr']].drop_duplicates()
from_addresses.columns = ['Display_Name', 'fk_Email_Addr']

# Extract unique 'To' addresses and display names
to_addresses = df_emails_imap[['To_Display_Name', 'To_Addr']].drop_duplicates()
to_addresses.columns = ['Display_Name', 'fk_Email_Addr']

# Combine the two DataFrames
df_email_addresses_dim = pd.concat([from_addresses, to_addresses]).drop_duplicates().reset_index(drop=True)

# Reorder columns to make 'fk_Email_Addr' the first column
cols = ['fk_Email_Addr'] + [col for col in df_email_addresses_dim.columns if col != 'fk_Email_Addr']
df_email_addresses_dim = df_email_addresses_dim[cols]

# Display the email addresses dimension
display(df_email_addresses_dim)

,fk_Email_Addr,Display_Name
0,do-not-reply@imdb.com,IMDb.com
1,notifications@updates.bandsintown.com,Bandsintown
2,english-personalized-digest@quora.com,Quora Digest
3,USPSInformeddelivery@email.informeddelivery.us...,USPS Informed Delivery
4,extracare@your.cvs.com,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=
...,...,...
510,Benemax.enrollment@onedigital.com,"'Deb Briere' <dbriere@interactions.com>, Benem..."
511,MIKE.CANCELL@GMAIL.COM,'MIKE.CANCELL@GMAIL.COM'
512,mikecancell@gmail.com,mikecancell
513,dbriere@interactions.com,Benemax Service <Benemax.service@onedigital.co...


### Add a Col for the Main Domain from the Addr

In [69]:
# Function to extract the main domain from an email address
def extract_main_domain(email):
    domain = email.split('@')[-1]
    main_domain = '.'.join(domain.split('.')[-2:])
    return main_domain

# Apply the function to the 'fk_Email_Addr' column
df_email_addresses_dim['Main_Domain'] = df_email_addresses_dim['fk_Email_Addr'].apply(extract_main_domain)

# Display the updated DataFrame
display(df_email_addresses_dim)

,fk_Email_Addr,Display_Name,Main_Domain
0,do-not-reply@imdb.com,IMDb.com,imdb.com
1,notifications@updates.bandsintown.com,Bandsintown,bandsintown.com
2,english-personalized-digest@quora.com,Quora Digest,quora.com
3,USPSInformeddelivery@email.informeddelivery.us...,USPS Informed Delivery,usps.com
4,extracare@your.cvs.com,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,cvs.com
...,...,...,...
510,Benemax.enrollment@onedigital.com,"'Deb Briere' <dbriere@interactions.com>, Benem...",onedigital.com
511,MIKE.CANCELL@GMAIL.COM,'MIKE.CANCELL@GMAIL.COM',GMAIL.COM
512,mikecancell@gmail.com,mikecancell,gmail.com
513,dbriere@interactions.com,Benemax Service <Benemax.service@onedigital.co...,interactions.com


### Save the Email Addresses to a Pickle for Caching

In [70]:
import pickle
import os

# Create the pickles directory if it does not exist
pickles_dir = '../.pickles'
os.makedirs(pickles_dir, exist_ok=True)

# Save the df_email_addresses_dim DataFrame to a pickle file
with open(f"{pickles_dir}/df_email_addresses_dim.pkl", 'wb') as f:
    pickle.dump(df_email_addresses_dim, f)

### Now from the Main Domain, Let's try to Get the Company Name from the Internet

#### Uses Parallelization for Performance

In [72]:
import whois
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time
import pandas as pd
import os
import pickle

# Load the existing df_email_addresses_dim DataFrame from the pickle file if it exists
pickle_path = '../.pickles/df_email_addresses_dim.pkl'
if os.path.exists(pickle_path):
    print("Loading existing df_email_addresses_dim DataFrame from pickle file...")
    with open(pickle_path, 'rb') as f:
        df_email_addresses_dim = pickle.load(f)
    print("DataFrame loaded successfully.")
else:
    print("Pickle file not found. A new pickle file will be created after data is retrieved.")
    # Initialize the DataFrame with necessary columns if pickle file is not found
    df_email_addresses_dim = df_email_addresses_dim.assign(
        Domain_Name='Unknown',
        Org_Name='Unknown',
        Name='Unknown'
    )

# Ensure the necessary columns are present in the DataFrame
required_columns = ['Domain_Name', 'Org_Name', 'Name']
for col in required_columns:
    if col not in df_email_addresses_dim.columns:
        df_email_addresses_dim[col] = 'Unknown'

# Function to get detailed information from main domain using whois with retry mechanism
def get_whois_info(main_domain, retries=3, delay=5):
    for attempt in range(retries):
        try:
            time.sleep(1.5)  # Add a delay before each whois call to address rate limiting
            domain_info = whois.whois(main_domain)
            return {
                'Domain_Name': domain_info.get('domain_name', 'Unknown'),
                'Org_Name': domain_info.get('org', 'Unknown'),
                'Name': domain_info.get('name', 'Unknown')
            }
        except Exception as e:
            print(f"Error resolving whois info for domain {main_domain} on attempt {attempt + 1}: {e}")
            time.sleep(delay)
    return {
        'Domain_Name': 'Unknown',
        'Org_Name': 'Unknown',
        'Name': 'Unknown'
    }

# Filter domains that need whois info
print("Filtering domains that need whois info...")
domains_to_process = df_email_addresses_dim[df_email_addresses_dim['Domain_Name'] == 'Unknown']['Main_Domain']
print(f"Found {len(domains_to_process)} domains to process.")
total_domains = len(df_email_addresses_dim)
skipped_domains = total_domains - len(domains_to_process)
print(f"Total domains: {total_domains}")
print(f"Skipped domains (already have WHOIS info): {skipped_domains}")

# Function to apply get_whois_info in parallel
def fetch_whois_info(domains):
    whois_info_list = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(tqdm(executor.map(get_whois_info, domains), total=len(domains), desc="Fetching WHOIS info"))
    for result in results:
        whois_info_list.append(result)
    return whois_info_list

# Apply the function to the filtered domains with progress indicator
print("Starting to fetch WHOIS info in parallel...")
whois_info = fetch_whois_info(domains_to_process)
print("WHOIS info fetching completed.")

# Convert the list of dictionaries to a DataFrame
print("Converting WHOIS info to DataFrame...")
whois_info_df = pd.DataFrame(whois_info, index=domains_to_process.index)
print("Conversion completed.")

# Update the existing DataFrame with the new whois info
print("Updating the existing DataFrame with the new WHOIS info...")
df_email_addresses_dim.update(whois_info_df)
print("DataFrame updated successfully.")

# Display the updated DataFrame
display(df_email_addresses_dim)

# Save the updated df_email_addresses_dim DataFrame to a pickle file
print("Saving the updated DataFrame to pickle file...")
with open(pickle_path, 'wb') as f:
    pickle.dump(df_email_addresses_dim, f)
print("DataFrame saved to pickle file successfully.")


Loading existing df_email_addresses_dim DataFrame from pickle file...
DataFrame loaded successfully.
Filtering domains that need whois info...
Found 515 domains to process.
Total domains: 515
Skipped domains (already have WHOIS info): 0
Starting to fetch WHOIS info in parallel...


Fetching WHOIS info:   0%|          | 0/515 [00:00<?, ?it/s]

Error resolving whois info for domain habitburger.com on attempt 1: [WinError 10054] An existing connection was forcibly closed by the remote host
Error resolving whois info for domain synchronyfinancial.com on attempt 1: [WinError 10054] An existing connection was forcibly closed by the remote host
Error resolving whois info for domain yahoo.com on attempt 1: [WinError 10054] An existing connection was forcibly closed by the remote host
Error resolving whois info for domain wellsfargo.com on attempt 1: [WinError 10054] An existing connection was forcibly closed by the remote host


Fetching WHOIS info:  64%|██████▍   | 332/515 [01:11<00:32,  5.58it/s]

Error resolving whois info for domain gumroad.com on attempt 1: [WinError 10054] An existing connection was forcibly closed by the remote host


Fetching WHOIS info: 100%|██████████| 515/515 [01:48<00:00,  4.73it/s]

WHOIS info fetching completed.
Converting WHOIS info to DataFrame...
Conversion completed.
Updating the existing DataFrame with the new WHOIS info...
DataFrame updated successfully.


,fk_Email_Addr,Display_Name,Main_Domain,Domain_Name,Org_Name,Name
0,do-not-reply@imdb.com,IMDb.com,imdb.com,IMDB.COM,"IMDb.com, Inc.",Unknown
1,notifications@updates.bandsintown.com,Bandsintown,bandsintown.com,BANDSINTOWN.COM,Identity Protection Service,On behalf of bandsintown.com owner
2,english-personalized-digest@quora.com,Quora Digest,quora.com,QUORA.COM,"Quora, Inc",Legal Team
3,USPSInformeddelivery@email.informeddelivery.us...,USPS Informed Delivery,usps.com,USPS.COM,US Postal Service,US Postal Service
4,extracare@your.cvs.com,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,cvs.com,CVS.COM,Unknown,Unknown
...,...,...,...,...,...,...
510,Benemax.enrollment@onedigital.com,"'Deb Briere' <dbriere@interactions.com>, Benem...",onedigital.com,ONEDIGITAL.COM,Digital Insurance LLC dba OneDigital,Domain Manager
511,MIKE.CANCELL@GMAIL.COM,'MIKE.CANCELL@GMAIL.COM',GMAIL.COM,GMAIL.COM,Unknown,Unknown
512,mikecancell@gmail.com,mikecancell,gmail.com,GMAIL.COM,Unknown,Unknown
513,dbriere@interactions.com,Benemax Service <Benemax.service@onedigital.co...,interactions.com,INTERACTIONS.COM,Unknown,Unknown


Saving the updated DataFrame to pickle file...
DataFrame saved to pickle file successfully.


### Now try to scrape the domain for meta data

In [77]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import pickle
import os

# Load the existing df_email_addresses_dim DataFrame from the pickle file if it exists
pickle_path = '../.pickles/df_email_addresses_dim.pkl'
if os.path.exists(pickle_path):
    with open(pickle_path, 'rb') as f:
        df_email_addresses_dim = pickle.load(f)
    print("DataFrame loaded successfully from pickle file.")
else:
    print("Pickle file not found. Please ensure the file exists and try again.")

# Add 'was_scraped' column if it doesn't exist
if 'was_scraped' not in df_email_addresses_dim.columns:
    df_email_addresses_dim['was_scraped'] = False

# Add meta data columns if they don't exist
meta_columns = ['og:site_name', 'og:title', 'og:description', 'og:url', 'og:image']
for col in meta_columns:
    if col not in df_email_addresses_dim.columns:
        df_email_addresses_dim[col] = 'Unknown'

# Function to get meta tag data with retry logic
def get_meta_data(domain_name, retries=3, delay=5):
    meta_data = {
        'og:site_name': 'Unknown',
        'og:title': 'Unknown',
        'og:description': 'Unknown',
        'og:url': 'Unknown',
        'og:image': 'Unknown'
    }
    for attempt in range(retries):
        try:
            url = f"http://{domain_name}"
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            for tag in meta_data.keys():
                meta_tag = soup.find('meta', attrs={'name': tag}) or soup.find('meta', attrs={'property': tag})
                if meta_tag and 'content' in meta_tag.attrs:
                    meta_data[tag] = meta_tag['content']
            return meta_data
        except Exception as e:
            print(f"Error fetching meta data for domain {domain_name} on attempt {attempt + 1}: {e}")
            time.sleep(delay)
    return meta_data

# Filter domains that need meta data (excluding 'Unknown' as valid data and not already scraped)
domains_to_process = df_email_addresses_dim[
    (df_email_addresses_dim['was_scraped'] == False)
]['Main_Domain']

print(f"Total domains to process: {len(domains_to_process)}")

# Function to apply get_meta_data in parallel
def fetch_meta_data(domains):
    meta_data_list = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(tqdm(executor.map(get_meta_data, domains), total=len(domains), desc="Fetching meta data"))
    for result in results:
        meta_data_list.append(result)
    return meta_data_list

# Apply the function to the filtered domains with progress indicator
meta_data = fetch_meta_data(domains_to_process)

# Convert the list of dictionaries to a DataFrame
meta_data_df = pd.DataFrame(meta_data, index=domains_to_process.index)

# Update the existing DataFrame with the new meta data
df_email_addresses_dim.update(meta_data_df)

# Set 'was_scraped' to True for the processed domains
df_email_addresses_dim.loc[domains_to_process.index, 'was_scraped'] = True

# Display the updated DataFrame
display(df_email_addresses_dim)

# Save the updated df_email_addresses_dim DataFrame to a pickle file
with open(pickle_path, 'wb') as f:
    pickle.dump(df_email_addresses_dim, f)

print("Meta data fetching and processing completed successfully.")
print(f"Total domains processed: {len(domains_to_process)}")
print(f"Total domains skipped (already had meta data): {skipped_domains}")
print(f"Updated DataFrame saved to pickle file: {pickle_path}")


DataFrame loaded successfully from pickle file.
Total domains to process: 0


Fetching meta data: 0it [00:00, ?it/s]


,fk_Email_Addr,Display_Name,Main_Domain,Domain_Name,Org_Name,Name,was_scraped,og:site_name,og:title,og:description,og:url,og:image
0,do-not-reply@imdb.com,IMDb.com,imdb.com,IMDB.COM,"IMDb.com, Inc.",Unknown,True,IMDb,"IMDb: Ratings, Reviews, and Where to Watch the...",IMDb is the world's most popular and authorita...,https://www.imdb.com/,https://m.media-amazon.com/images/G/01/imdb/im...
1,notifications@updates.bandsintown.com,Bandsintown,bandsintown.com,BANDSINTOWN.COM,Identity Protection Service,On behalf of bandsintown.com owner,True,Unknown,Unknown,Unknown,Unknown,Unknown
2,english-personalized-digest@quora.com,Quora Digest,quora.com,QUORA.COM,"Quora, Inc",Legal Team,True,Quora,Quora,,https://www.quora.com/,https://qsf.cf2.quoracdn.net/-4-images.share_d...
3,USPSInformeddelivery@email.informeddelivery.us...,USPS Informed Delivery,usps.com,USPS.COM,US Postal Service,US Postal Service,True,Unknown,Welcome | USPS,"Welcome to USPS.com. Track packages, pay and p...",https://www.usps.com/,https://www.usps.com/assets/images/welcome/usp...
4,extracare@your.cvs.com,=?UTF-8?B?Q1ZTIEV4dHJhQ2FyZQ==?=,cvs.com,CVS.COM,Unknown,Unknown,True,Unknown,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
510,Benemax.enrollment@onedigital.com,"'Deb Briere' <dbriere@interactions.com>, Benem...",onedigital.com,ONEDIGITAL.COM,Digital Insurance LLC dba OneDigital,Domain Manager,True,OneDigital,"OneDigital | Advocates of Health, Success and ...","Access insurance, financial services, and HR c...",https://www.onedigital.com/,https://www.onedigital.com/wp-content/uploads/...
511,MIKE.CANCELL@GMAIL.COM,'MIKE.CANCELL@GMAIL.COM',GMAIL.COM,GMAIL.COM,Unknown,Unknown,True,Unknown,Unknown,Unknown,Unknown,Unknown
512,mikecancell@gmail.com,mikecancell,gmail.com,GMAIL.COM,Unknown,Unknown,True,Unknown,Unknown,Unknown,Unknown,Unknown
513,dbriere@interactions.com,Benemax Service <Benemax.service@onedigital.co...,interactions.com,INTERACTIONS.COM,Unknown,Unknown,True,Interactions,Interactions IVA | Conversational AI Solutions...,"Improve CX, reduce costs. Our Intelligent Virt...",https://www.interactions.com/,Unknown


Meta data fetching and processing completed successfully.
Total domains processed: 0
Total domains skipped (already had meta data): 0
Updated DataFrame saved to pickle file: ../.pickles/df_email_addresses_dim.pkl


### Resave the Pickle

In [54]:
# Save the updated df_email_addresses_dim DataFrame to a pickle file
with open('../.pickles/df_email_addresses_dim.pkl', 'wb') as f:
    pickle.dump(df_email_addresses_dim, f)

## Create a Fact Table

In [15]:
# Merge the email addresses dimension with the main DataFrame on 'From_Addr' and 'To_Addr'
fact_table = df_emails_imap.merge(df_email_addresses_dim, left_on='From_Addr', right_on='fk_Email_Addr', how='left')
fact_table = fact_table.merge(df_email_addresses_dim, left_on='To_Addr', right_on='fk_Email_Addr', how='left', suffixes=('_From', '_To'))

# Merge the datetime dimension with the main DataFrame on 'Timestamp'
fact_table = fact_table.merge(df_datetime_dim, left_on='Timestamp', right_on='fk_Timestamp', how='left')

# Select relevant columns for the fact table
fact_table = fact_table[['Timestamp', 'Subject', 'Message-ID', 'From_Addr', 'To_Addr', 'Body_Text', 'Email_Category']]

# Display the fact table
display(fact_table)

,Timestamp,Subject,Message-ID,From_Addr,To_Addr,Body_Text,Email_Category
0,2024-12-01 08:19:12+00:00,See what's streaming in December,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,do-not-reply@imdb.com,mike.cancell@gmail.com,IMDb Streaming Guides\r\nCheck out our watch g...,Entertainment
1,2024-12-01 08:19:12+00:00,See what's streaming in December,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,do-not-reply@imdb.com,mike.cancell@gmail.com,IMDb Streaming Guides\r\nCheck out our watch g...,Entertainment
2,2024-12-01 08:19:12+00:00,See what's streaming in December,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,do-not-reply@imdb.com,mike.cancell@gmail.com,IMDb Streaming Guides\r\nCheck out our watch g...,Entertainment
3,2024-12-01 08:19:12+00:00,See what's streaming in December,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,do-not-reply@imdb.com,mike.cancell@gmail.com,IMDb Streaming Guides\r\nCheck out our watch g...,Entertainment
4,2024-12-01 08:19:12+00:00,See what's streaming in December,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,do-not-reply@imdb.com,mike.cancell@gmail.com,IMDb Streaming Guides\r\nCheck out our watch g...,Entertainment
...,...,...,...,...,...,...,...
25415,2025-01-27 20:53:49+00:00,Next Steps: AWS Rapid Ramp Credits Application,<01000194a98abc74-6b3ed49b-a99f-4934-9dc3-83a3...,ablelnat@amazon.com,mike.cancell@gmail.com,"Hi Michael,\r\n\r\n\r\nThis is Natty with AWS....",Meeting
25416,2025-01-27 20:53:49+00:00,Next Steps: AWS Rapid Ramp Credits Application,<01000194a98abc74-6b3ed49b-a99f-4934-9dc3-83a3...,ablelnat@amazon.com,mike.cancell@gmail.com,"Hi Michael,\r\n\r\n\r\nThis is Natty with AWS....",Meeting
25417,2025-01-27 20:53:49+00:00,Next Steps: AWS Rapid Ramp Credits Application,<01000194a98abc74-6b3ed49b-a99f-4934-9dc3-83a3...,ablelnat@amazon.com,mike.cancell@gmail.com,"Hi Michael,\r\n\r\n\r\nThis is Natty with AWS....",Meeting
25418,2025-01-27 20:53:49+00:00,Next Steps: AWS Rapid Ramp Credits Application,<01000194a98abc74-6b3ed49b-a99f-4934-9dc3-83a3...,ablelnat@amazon.com,mike.cancell@gmail.com,"Hi Michael,\r\n\r\n\r\nThis is Natty with AWS....",Meeting


### Clean the Fact Table

In [16]:
# Get the initial count of rows in the fact table
initial_count = fact_table.shape[0]

# Drop duplicates from the fact table
fact_table.drop_duplicates(inplace=True)

# Get the final count of rows in the fact table
final_count = fact_table.shape[0]

# Calculate the number of dropped rows
dropped_count = initial_count - final_count

# Display the count of dropped rows
print(f"Number of dropped rows: {dropped_count}")

# Display the updated fact table
display(fact_table)

Number of dropped rows: 24294


,Timestamp,Subject,Message-ID,From_Addr,To_Addr,Body_Text,Email_Category
0,2024-12-01 08:19:12+00:00,See what's streaming in December,<01000193814d60c2-fd528199-2ca9-40dc-8c71-c310...,do-not-reply@imdb.com,mike.cancell@gmail.com,IMDb Streaming Guides\r\nCheck out our watch g...,Entertainment
11,2024-12-01 08:51:40+00:00,Coming soon to State Theatre New Jersey: Cirqu...,<msg-19742805299@email.bandsintown.com>,notifications@updates.bandsintown.com,mike.cancell@gmail.com,"Just Announced Cirque Musica, New Jersey Symph...",Music
22,2024-12-01 12:03:31+00:00,=?utf-8?q?Will_Obama_go_to_jail=3F?=,<LARkCC7mJXguO-fXuuzZcwMdZJYRZCZHc2nstcrUwcMNj...,english-personalized-digest@quora.com,mike.cancell@gmail.com,Top stories for Mike\r\n\r\n-----\r\n\r\nQuest...,Newsletter
33,2024-12-01 12:27:32+00:00,"Your Daily Digest for Sun, 12/1 is ready to view",<20241201122732.9884238d69c03837@email.informe...,USPSInformeddelivery@email.informeddelivery.us...,mike.cancell@gmail.com,Content-Type: text/plain; charset=UTF-8\r\nCon...,Shipping
44,2024-12-01 13:10:38+00:00,Your Weekly Ad + Holiday Savings to Deck the H...,<0.3.9.54A.1DB43F26A5DD630.0@omp.your.cvs.com>,extracare@your.cvs.com,mike@mikecancell.com,CVS\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,Shipping
...,...,...,...,...,...,...,...
25373,2025-01-27 18:31:14+00:00,RE: Interactions - Michael Cancell,<SN7PR20MB5920F391F342A45235E3FF4097EC2@SN7PR2...,Benemax.service@onedigital.com,mike@mikecancell.com,"Hi Mike,\r\n\r\nThe Rx reimbursement processed...",Job Application
25380,2025-01-27 19:51:19+00:00,Update from Deque Systems,<c449ad01-c532-4f18-9ceb-5d415e295c10@mtasv.net>,recruiting+335397270@applytojob.com,mike@mikecancell.com,"Hello Michael,Thank you for the time and energ...",Other
25387,2025-01-27 20:18:01+00:00,Thank you for applying,<01000194a969f744-85025198-21e5-4440-a268-1c22...,aws-marketing-email-replies@amazon.com,mike.cancell@gmail.com,Thank you for your interest in the $300 AWS cr...,Job Rejection
25398,2025-01-27 20:18:30+00:00,"Follow your application status, Mike",<u6w8VwXDpwNOIRXHsa0jc4qtKza1Awt3@siemens.avat...,talentacquisition.hr@siemens.com,mike.cancell@gmail.com,"Hi Mike,\r\n\r\nThank you for your interest in...",Other


# Save the DFs as Pickles

### Check the DFs that Exist

In [17]:
# List all DataFrames
dataframes = [var for var in globals() if isinstance(globals()[var], pd.DataFrame)]
print(dataframes)

['df_emails_imap', 'df_datetime_dim', 'from_addresses', 'to_addresses', 'df_email_addresses_dim', 'whois_info_df', 'fact_table']


### Only Save the Ones we Really Need

In [18]:
import pickle
import os

# Create the pickles directory if it does not exist
pickles_dir = '../.pickles'
os.makedirs(pickles_dir, exist_ok=True)

# List of DataFrame names
df_names = ['df_datetime_dim', 'df_email_addresses_dim', 'fact_table']

# Loop through the list and store each DataFrame if it exists
for name in df_names:
    if name in globals():
        with open(f"{pickles_dir}/{name}.pkl", 'wb') as f:
            pickle.dump(globals()[name], f)
    else:
        print(f"DataFrame {name} does not exist and will not be pickled.")
